# Generate Data for Chartbook

Brian Dew

@bd_econ

In [4]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### S&P 500 

In [2]:
ltdate = int(time.time())

url = ('https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?'+
       f'period1=599616000&period2={ltdate}&interval=1d&events=history')

df = pd.read_csv(url).set_index('Date')

close = df['Adj Close']

close.index = pd.to_datetime(close.index)

final = close.resample('MS').mean().append(pd.Series({close.index[-1]: close[-1]}))
final.name = 'GSPC'
final.to_csv(data_dir / 'sp500.csv', index_label='date', float_format='%g') 

data = pd.DataFrame()
data['GSPC'] = close.loc['2017':]
data['MA'] = close.rolling(253).mean().loc['2017':]
data.to_csv(data_dir / 'sp500_recent.csv', index_label='date', float_format='%g')

In [3]:
ltdate = dtxt(close.index[-1])['day1']
ltval = close.iloc[-1]
hival = df.High.iloc[-253:].max()
chhi = (1 - ltval/hival) * 100
hidt = dtxt(df.High.iloc[-253:].idxmax())['day1']
loval = df.Low.iloc[-253:].min()
chlo = abs((1 - ltval/loval) * 100)
lodt = dtxt(df.Low.iloc[-253:].idxmin())['day1']
avval = df['Adj Close'].iloc[-253:].mean()
chav = (1 - ltval/avval) * 100
chavabs = abs(chav)
chtype = 'below' if chav > 0 else 'above'

text = (f'The S\&P 500 closed at {ltval:.0f} on {ltdate}. The index is currently '+
        f'{chhi:.1f} percent below its one-year high of {hival:.0f} on {hidt}, and '+
        f'{chlo:.1f} percent above its one-year low of {loval:.0f} on {lodt}. The '+
        f'average over the past year is {avval:.0f}; the index is {chavabs:.1f} percent '+
        f'{chtype} its one-year moving average '+
         '(see {\color{black!18!white}\\textbf{---}}).')

write_txt(text_dir / 'sp500.txt', text)

text

'The S\\&P 500 closed at 3130 on July 2, 2020. The index is currently 7.8 percent below its one-year high of 3394 on February 19, 2020, and 42.8 percent above its one-year low of 2192 on March 23, 2020. The average over the past year is 3008; the index is 4.1 percent above its one-year moving average (see {\\color{black!18!white}\\textbf{---}}).'

### Interest rates

In [4]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=2d30c0abba5db9568b0bfc379ecec978&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

n = {'RIFLGFCY10_N.B': 'Ten-year',
    'RIFLGFCY30_N.B': 'Thirty-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY05_N.B': 'Five-year',
    'RIFLGFCY02_N.B': 'Two-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('M').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

In [5]:
ldate = dtxt(data.index[-1])['day1']
tenlt = data['Ten-year'].iloc[-1]
tenpr = data['Ten-year'].iloc[-13]
twolt = data['Two-year'].iloc[-1]
twopr = data['Two-year'].iloc[-13]

text = (f'As of {ldate}, the constant maturity yield for a ten-year '+
        'Treasury bond (see {\color{red!70!purple!90!black}\\textbf{---}}) '+
        f'is {tenlt} percent, compared to {tenpr:.2f} percent one year prior. '+
        'The yield for a two-year Treasury (see {\color{orange!80!yellow}\\textbf{---}}) '
        +f'is {twolt} percent, compared to {twopr:.2f} '+
        f'percent a year prior.')

write_txt(text_dir / 'rates_basic.txt', text)

print(text)

text = ('The effective fed funds rate (see {\color{blue!60!black}\\textbf{---}}) '+
        f'is {data["Fed Funds"].iloc[-1]} percent, as of {ldate}.')

write_txt(text_dir / 'rates_ff.txt', text)

text

As of July 1, 2020, the constant maturity yield for a ten-year Treasury bond (see {\color{red!70!purple!90!black}\textbf{---}}) is 0.69 percent, compared to 2.06 percent one year prior. The yield for a two-year Treasury (see {\color{orange!80!yellow}\textbf{---}}) is 0.17 percent, compared to 1.84 percent a year prior.


'The effective fed funds rate (see {\\color{blue!60!black}\\textbf{---}}) is 0.08 percent, as of July 1, 2020.'

### Spread

In [6]:
data = df[df['Ten-year'] != 'ND'].astype('float')
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2012':].to_csv(data_dir / 'spread.csv', index_label='date', float_format='%g', header=True)

node = end_node(spread['Ten-3M'], 'cyan!60!blue').replace('\\%', '')
write_txt(text_dir / 'spread_node.txt', node)

node = end_node(spread['Ten-2Y'], 'orange').replace('\\%', '')
write_txt(text_dir / 'spread_node2.txt', node)

lt3 = spread['Ten-3M'].iloc[-1]
lt3t = f'{lt3:.2f} {["percentage points" if lt3 > 1.05 else "percentage point"][0]}'
pr3 = spread['Ten-3M'].iloc[-252]
pr3t = f'{pr3:.2f} {["percentage points" if pr3 > 1.05 else "percentage point"][0]}'
lt2 = spread['Ten-2Y'].iloc[-1]
lt2t = f'{lt2:.2f} {["percentage points" if lt2 > 1.05 else "percentage point"][0]}'
pr2 = spread['Ten-2Y'].iloc[-252]
pr2t = f'{pr2:.2f} {["percentage points" if pr2 > 1.05 else "percentage point"][0]}'

text = (f'As of {ldate}, the spread between a 10-year treasury bond and '+
        f'a three-month treasury bill is {lt3t} '+
        '(see {\color{cyan!60!blue}\\textbf{---}}), compared to '+
        f'{pr3t} one year prior. The spread between 10-year '+
        'and 2-year treasuries (see {\color{orange}\\textbf{---}}) '+
        f'was {lt2t} on {ldate}, and '+
        f'{pr2t} one year prior.')

write_txt(text_dir / 'spread_basic.txt', text)

text

/home/brian/miniconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


'As of July 1, 2020, the spread between a 10-year treasury bond and a three-month treasury bill is 0.55 percentage point (see {\\color{cyan!60!blue}\\textbf{---}}), compared to -0.18 percentage point one year prior. The spread between 10-year and 2-year treasuries (see {\\color{orange}\\textbf{---}}) was 0.52 percentage point on July 1, 2020, and 0.25 percentage point one year prior.'

### Gold Price

In [7]:
df = fred_df('GOLDAMGBD228NLBM').dropna()
data = (df.resample('M').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'gold.csv', index_label='date')

ltdate = dtxt(data.index[-1])['day1']
ltval = data.VALUE.iloc[-1]
prdate = dtxt(data.index[-13])['mon1']
prval = data.VALUE.iloc[-13]
grdate = dtxt(data.loc['2006':'2011', 'VALUE'].idxmax())['mon1']
grval = data.loc['2006':'2011', 'VALUE'].max()

text = (f'As of {ltdate}, one troy ounce of gold sells for \${ltval:,.2f} '+
        '(see {\color{orange!40!yellow}\\textbf{---}}), '
        f'compared to an average of \${prval:,.2f} per ounce during {prdate}. '+
        'Following the great recession, the monthly average price '+
        f'of gold reached \${grval:,.2f} per ounce, in {grdate}.')
print(text)
write_txt(text_dir / 'gold.txt', text)

As of July 2, 2020, one troy ounce of gold sells for \$1,771.85 (see {\color{orange!40!yellow}\textbf{---}}), compared to an average of \$1,414.61 per ounce during July 2019. Following the great recession, the monthly average price of gold reached \$1,780.65 per ounce, in September 2011.


### Fed Balance Sheet

In [8]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=38d757c01cb3b550176f371352643679&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data = clean_data / 1_000_000

clean_data.to_csv(data_dir / 'fed_assets.csv', index_label='date', float_format='%g')

In [9]:
ltval = clean_data['RESPPMA_N.WW'].iloc[-1].round(1)
ltdate = dtxt(clean_data.index[-1])['day1']
n = (clean_data.iloc[-1] - clean_data.iloc[-2])['RESPPMA_N.WW'] * 1000
chval = (f'increased by \${abs(n):.0f} billion' if n >= 10 
         else f'decreased by \${abs(n):.0f} billion' 
         if n <= -10 else 'was largely unchanged')

text = ('In response to the collapse of the housing bubble, the Fed purchased '+
        'U.S. Treasury bonds and mortgage-backed securities. Total assets held by '+
        'the Federal Reserve (see {\color{blue!80!black}\\textbf{---}}) '+
        'increased from \$0.9 trillion in August 2008 to '+
        '\$2.2 trillion in November 2008. Additional rounds of asset purchases, '+
        'referred to as quantitative easing, increased the balance sheet to '+
        '\$4.5 trillion by January 2014. As bonds mature they were replaced '+
        'until October 2017, when the Fed allowed the size of its balance '+
        'sheet to normalize. Total assets fell below \$3.8 trillion in August 2019.')

write_txt(text_dir / 'fed_assets1.txt', text)
        
        
txt2 = ('Balance sheet normalization ended in September 2019 when the Fed increased '+
        'operations in overnight and term repurchase agreement (repo) markets, following '+
        'a sharp increase in rates in these markets. The Fed balance sheet '+
        'increased to \$4.1 trillion by December 2019. More recently, in response to '+
        'worsening financial conditions, the Fed began to purchase commercial bonds '+
        'and to offer currency swaps with major US trading partners. '+
        'The Fed balance sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. Fed total assets '+
        f'{chval} over the previous week value.')

write_txt(text_dir / 'fed_assets2.txt', txt2)

print(txt2)

Balance sheet normalization ended in September 2019 when the Fed increased operations in overnight and term repurchase agreement (repo) markets, following a sharp increase in rates in these markets. The Fed balance sheet increased to \$4.1 trillion by December 2019. More recently, in response to worsening financial conditions, the Fed began to purchase commercial bonds and to offer currency swaps with major US trading partners. The Fed balance sheet increased from \$4.2 trillion in February 2020 to \$7.0 trillion, as of the latest data, covering July 1, 2020. Fed total assets decreased by \$73 billion over the previous week value.


### Fed Balance Sheet Table

In [10]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=bbb7b50f663fe933c8cce7e3707e3998&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d2 = {'RESPPA_N.WW': '\\textbf{Total} (see {\color{blue!80!black}\\textbf{---}})',
      'RESPPALGUO_N.WW': '\hspace{2mm}U.S. Treasury securities',
      'RESPPALGASMO_N.WW': '\hspace{2mm}Mortgage-backed securities',
      'RESH4SCS_N.WW': '\hspace{2mm}Central bank liquidity swaps',
      'RESPPALGTR_N.WW': '\hspace{2mm}Repurchase agreements',
      'RESPPALD_N.WW': '\hspace{2mm}Loans',
      'RESPPALDJ_N.WW': '\hspace{4mm}Payroll Protection Program',
      'NetPremDisc': '\hspace{2mm}Net unamortized premium',
      'Other': '\hspace{2mm}Other'}

pr_di = ['RESPPALSD_N.WW', 'RESPPALSP_N.WW']

data = clean_data.copy()
data['Other'] = (data['RESPPA_N.WW'] - 
                             data.drop('RESPPA_N.WW', axis=1).sum(axis=1))
data['NetPremDisc'] = data[pr_di].sum(axis=1)

data = data.rename(d2, axis=1).drop(pr_di, axis=1)

df = pd.DataFrame()

for i in [-1, -2, -5, -14, -53]:
    df[dtxt(data.index[i])['day2']] = data.iloc[i]
    
(df.loc[list(d2.values()), :].div(1000)
   .applymap('{:,.1f}'.format)
   .to_csv(data_dir / 'fed_bal_sheet.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

### Money Growth

In [11]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H6&series=fafc1295c552e99d2b907eb62278e4ca&lastobs=&'
dt = 'from=01/01/1988&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

final2 = {}
data = clean_data[['M2_N.WM', 'MMFIN_N.WM']].sum(axis=1)
month_list = clean_data.resample('MS').mean().pct_change(12).dropna().index
short_month = ''
for date in month_list:
    month_len = len(data.loc[date.strftime('%Y-%m')])
    prevyr = f'{date.year - 1}-{date.month}'
    weeks_in_short_month = 4
    end_date = date
    if month_len < 4:
        end_date = date
        short_month = date.strftime('%B %Y')
        val = data.loc[date.strftime('%Y-%m')].mean()
        prv = data.loc[prevyr].iloc[:month_len]
        prev = prv.mean()
        weeks_in_short_month = len(prv)
    elif month_len >= 4:
        val = data.loc[date.strftime('%Y-%m')].mean()
        prev = data.loc[prevyr].mean()
        end_full = date
    final2[date] = (val / prev - 1) * 100
    
final = pd.Series(final2, name='value')
final.to_csv(data_dir / 'm2imf.csv', index_label='date', header='True')

s = series_info(final)

node = end_node(final, 'green!80!blue')
write_txt(text_dir / 'm2imf_node.txt', node)

week_conv = {1: f'the first week of {short_month}', 
             2: f'the first two weeks of {short_month}', 
             3: f'the first three weeks of {short_month}',
             4: end_full.strftime('%B %Y')}

if final.iloc[-1] >= 0.1:
    txt = f'increased over the equivalent previous year value by {final.iloc[-1]:.1f} percent'
elif final.iloc[-1] <= -0.1:
    txt = f'decreased over the equivalent previous year value by {abs(final.iloc[-1]):.1f} percent'   
else:
    txt = 'was virtually unchanged over the previous year value'
    
if s['days_since_match'] > 300:
    txt2 = f", {s['last_matched'].replace('highest level', 'fastest growth rate')}."
else:
    txt2 = '.'
    
text = (f'In {week_conv[weeks_in_short_month]}, '+
        f'the M2 plus institutional money funds measure {txt}{txt2}')

write_txt(text_dir / 'm2imf.txt', text)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2sh = (clean_data['M2_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100
imfsh = (clean_data['MMFIN_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100

text2 = (f'In the week of {clean_data.index[-1].strftime("%B %-d, %Y")}, '+
         f'the M2 measure of money averaged \${clean_data["M2_N.WM"].iloc[-1] / 1000:.1f} '+
         f'trillion, equivalent to {m2sh:.1f} percent of GDP. Institution money market '
         +f'accounts, which are not included in M2, can be combined with M2 to create a '+
         f'slightly-broader-than-M2 measure of the money stock. These funds averaged '+
         f'\${clean_data["MMFIN_N.WM"].iloc[-1] / 1000:.1f} trillion in the same week, '+
         f'equivalent to {imfsh:.1f} percent of GDP. ')

write_txt(text_dir / 'm2imf2.txt', text2)
print(text, '\n')
print(text2)

In June 2020, the M2 plus institutional money funds measure increased over the equivalent previous year value by 28.4 percent. 

In the week of June 22, 2020, the M2 measure of money averaged \$18.3 trillion, equivalent to 85.0 percent of GDP. Institution money market accounts, which are not included in M2, can be combined with M2 to create a slightly-broader-than-M2 measure of the money stock. These funds averaged \$3.2 trillion in the same week, equivalent to 14.8 percent of GDP. 


### Exchange rates

In [7]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H10&series=ad1712193ad5bad7b424e3ae5eb101a5&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data['RXI_N.B.EU'] = 1 / clean_data['RXI$US_N.B.EU'] 
clean_data['RXI_N.B.UK'] = 1 / clean_data['RXI$US_N.B.UK'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
oth = ['RXI_N.B.MX', 'RXI_N.B.BZ', 'RXI_N.B.CH', 'RXI_N.B.SD']
twidx = clean_data.resample('MS').mean().append(latest)['JRXWTFB_N.B'].dropna()
clean_data.resample('MS').mean().append(latest)[major].to_csv(data_dir / 'fx1.csv', index_label='date', float_format='%g')
clean_data.resample('MS').mean().append(latest)[oth].to_csv(data_dir / 'fx2.csv', index_label='date', float_format='%g')
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', float_format='%g', header=True)

ldate = dtxt(twidx.index[-1])['day1']
lval = twidx.iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx.iloc[-38:].mean()
prev3yr = twidx.iloc[-74:-38].mean()

text = (f'The latest index value, as of {ldate}, is {lval:.1f}, an increase '+
        f'of {totch:.1f} percent since inception in 2006. Over the past three years, '+
        f'the index value has averaged {threeyr:.1f}, compared to an average '+
        f'of {prev3yr:.1f} over the previous three-year period.')

write_txt(text_dir / 'twdbasic.txt', text)

text

'The latest index value, as of June 26, 2020, is 120.9, an increase of 20.9 percent since inception in 2006. Over the past three years, the index value has averaged 114.4, compared to an average of 108.6 over the previous three-year period.'

In [8]:
cad = clean_data.loc[ldate, 'RXI_N.B.CA']
jpy = clean_data.loc[ldate, 'RXI_N.B.JA'] * 100
eur = clean_data.loc[ldate, 'RXI_N.B.EU']
gbp = clean_data.loc[ldate, 'RXI_N.B.UK']
cadpc = inc_dec_percent(clean_data['RXI_N.B.CA'].pct_change(262).iloc[-1] * 100)
jpypc = inc_dec_percent(clean_data['RXI_N.B.JA'].pct_change(262).iloc[-1] * 100)
gbppc = inc_dec_percent(clean_data['RXI_N.B.UK'].pct_change(262).iloc[-1] * 100)
eurpc = inc_dec_percent(clean_data['RXI_N.B.EU'].pct_change(262).iloc[-1] * 100)


text = (f'As of {ldate}, one US dollar buys approximately: {cad:.2f} Canadian dollars '+
        f'(see {{\color{{green!85!blue}}\\textbf{{---}}}}), {jpy:.0f} Japanese Yen '+
        f'(see {{\color{{red}}\\textbf{{---}}}}), {eur:.2f} Euros '+
        f'(see {{\color{{cyan!90!white}}\\textbf{{---}}}}), and {gbp:.2f} British Pounds '+
         '(see {\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal '+
        f'exchange rate between the US dollar and the Canadian dollar {cadpc}, the USD-JPY rate '+
        f'{jpypc}, the USD-EUR rate {eurpc}, and the USD-GBP rate {gbppc}.')

write_txt(text_dir / 'selcurr_basic.txt', text)

text

'As of June 26, 2020, one US dollar buys approximately: 1.37 Canadian dollars (see {\\color{green!85!blue}\\textbf{---}}), 107 Japanese Yen (see {\\color{red}\\textbf{---}}), 0.89 Euros (see {\\color{cyan!90!white}\\textbf{---}}), and 0.81 British Pounds (see {\\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar increased by 4.5 percent, the USD-JPY rate decreased by 0.4 percent, the USD-EUR rate increased by 1.6 percent, and the USD-GBP rate increased by 2.9 percent.'

### Initial jobless claims

In [2]:
data = fred_df('ICNSA', start='2014') / 1000 

data['1M'] = data['VALUE'].rolling(4).mean()

data.div(1000).to_csv(data_dir / 'icsa.csv', index_label='date', float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest1m = data["1M"].iloc[-1]*1000
prev1m = data["1M"].iloc[-53]*1000

chval = totval - data['VALUE'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = ('The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reported}} '+
        f'{totval:,.0f} actual new claims for unemployment '+
        'insurance (UI; see {\color{cyan!85!blue}\\textbf{---}}) during the week ending '+
        f'{datelt}, {change}. Over the past four weeks, '+
        f'new claims have averaged {round(latest1m,-2):,.0f} per week. During the same four-'+
        f'week period last year, there were an average of {round(prev1m,-2):,.0f} new claims per week.')

write_txt(text_dir / 'icsa.txt', text)

text

'The Department of Labor \\href{{https://www.dol.gov/ui/data.pdf}}{{reported}} 1,445,481 actual new claims for unemployment insurance (UI; see {\\color{cyan!85!blue}\\textbf{---}}) during the week ending June 27, 2020, a one-week decrease of 14,600. Over the past four weeks, new claims have averaged 1,482,500 per week. During the same four-week period last year, there were an average of 219,100 new claims per week.'

In [6]:
data = fred_df('CCNSA', start='2014') / 1000 
data.div(1000).to_csv(data_dir / 'ccsa.csv', index_label='date', float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
prevval = data['VALUE'].iloc[-53]*1000
datepr = dtxt(data.index[-53])['day1']

chval = totval - data['VALUE'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = (f'For the week ending {datelt}, the Department of Labor '+
        '\href{{https://www.dol.gov/ui/data.pdf}}{{reports}} '+
        f'{totval:,.0f} continued claims for unemployment '+
        'insurance (insured unemployed, see {\color{green!70!blue}\\textbf{---}})'+
        f', {change}. One year prior, during the week of {datepr}, '+
        f'there were an average of {round(prevval,-2):,.0f} '+
        'insured unemployed.')

write_txt(text_dir / 'ccsa.txt', text)

text

'For the week ending June 20, 2020, the Department of Labor \\href{{https://www.dol.gov/ui/data.pdf}}{{reports}} 17,920,654 continued claims for unemployment insurance (insured unemployed, see {\\color{green!70!blue}\\textbf{---}}), a one-week increase of 266,400. One year prior, during the week of June 22, 2019, there were an average of 1,588,900 insured unemployed.'

### VIX (SP500 volatility)

In [12]:
prev = pd.read_excel(data_dir/ 'vixarchive.xls', skiprows=1, index_col='Date')
VIX = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
curr = pd.read_csv(VIX, skiprows=1, index_col='Date', parse_dates=True)
df = (prev.append(curr)['VIX Close'].resample('MS').mean()
      .append(curr['VIX Close'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')

node = end_node(df, 'magenta', percent=False)
write_txt(text_dir / 'vix_node.txt', node)

ldate = dtxt(df.index[-1])['day1']
vallt = df.iloc[-1]
val3y = df.iloc[-37:].mean()

compare = compare_text(vallt, val3y, [3, 15, 30])

text = ('This volatility measure, the VIX index (see {\color{magenta}\\textbf{---}}), '+
        f'was {vallt:.1f} on {ldate}, '+
        f'{compare} the average index value of {val3y:.1f} over the '+
        'past three years.')

write_txt(text_dir / 'vixbasic.txt', text)

text

'This volatility measure, the VIX index (see {\\color{magenta}\\textbf{---}}), was 27.7 on July 2, 2020, slightly above the average index value of 18.6 over the past three years.'

### Oil prices (WTI)

In [13]:
df = fred_df('DCOILWTICO')

latest = df.iloc[-1]

ma = df.resample('MS').mean()

p = ma.append(latest)['VALUE']

(p.to_csv(data_dir / 'wti.csv', index_label='date', header=True))

oneyr = p.pct_change(13).iloc[-1] * 100

threeyr = p.pct_change(37).iloc[-1] * 100

oyt = inc_dec_percent(oneyr)
tyt = inc_dec_percent(threeyr)
    
ltch = p.loc['2008-06-01'] - p.iloc[-1]

ldate = dtxt(p.index[-1])['day1']

text = (f'As of {ldate}, a barrel of west Texas '+
        f'intermediate (WTI) \\textbf{{crude oil}} sells for \${p.iloc[-1]:.2f} '+
        '(see {\color{red!80!purple}\\textbf{---}}). Over the past '+
        f'year, this measure of oil prices has {oyt}. Over the past three years, '+
        f'the price {tyt}. Currently, the WTI price is \${ltch:.2f} per barrel '+
        'below its June 2008 average.')

write_txt(text_dir / 'wti.txt', text)

text

'As of June 29, 2020, a barrel of west Texas intermediate (WTI) \\textbf{crude oil} sells for \\$39.67 (see {\\color{red!80!purple}\\textbf{---}}). Over the past year, this measure of oil prices has decreased by 27.4 percent. Over the past three years, the price decreased by 12.2 percent. Currently, the WTI price is \\$94.21 per barrel below its June 2008 average.'

### Inflation Expectations

In [14]:
url = 'http://www.sca.isr.umich.edu/files/tbmpx1px5.csv'
df = pd.read_csv(url)
df.index = pd.to_datetime(df['Month'] + ' ' + df['YYYY'].astype('str'))
data = df['PX5_MD'].loc['2015':]

data.to_csv(data_dir / 'infumich.csv', index_label='date', header='VALUE')

node = end_node(data, 'violet')
write_txt(text_dir / 'infumich_node.txt', node)

data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', header=True)

node = end_node(data1, 'blue!70!black')
write_txt(text_dir / 'infbreak_node.txt', node)

ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-13])['mon1']
p5val = data.iloc[-61]
lval = data.iloc[-1]
pval = data.iloc[-13]

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.index[-262])['day1']
pvalm = data1.iloc[-262]
p5valm = data1.iloc[-(262*5)]

inf_act = pd.read_csv(data_dir / 'cpi.csv')['ALL'].iloc[-60:].mean()

text = (f'As of {ldate}, consumers expect an average inflation rate of {lval} '+
        'percent over the next five years, (see {\color{violet!60!magenta}\\textbf{---}}), '+
        f'compared to an expected rate of {pval} percent '+
        f'in {pdate}. Consumers had expected inflation to average {p5val} percent over the past '+
        f'five years, while actual inflation over the period was {inf_act:.1f} percent.')

write_txt(text_dir / 'inf_exp_cons.txt', text)

print(text)

text2 = (f'As of {ldatem}, markets expect an average inflation rate of {lvalm} percent ' + 
         'over the next five years (see {\color{blue!70!black}\\textbf{---}}), '
         +f'compared to an expected rate of {pvalm} percent on '+
         f'{pdatem}. Markets had expected inflation to average {p5valm} percent over the '+
         f'past five years, five years ago.')

write_txt(text_dir / 'inf_exp_mkts.txt', text2)

print(text2)

As of June 2020, consumers expect an average inflation rate of 2.5 percent over the next five years, (see {\color{violet!60!magenta}\textbf{---}}), compared to an expected rate of 2.3 percent in June 2019. Consumers had expected inflation to average 2.6 percent over the past five years, while actual inflation over the period was 1.7 percent.
As of July 2, 2020, markets expect an average inflation rate of 1.21 percent over the next five years (see {\color{blue!70!black}\textbf{---}}), compared to an expected rate of 1.51 percent on July 3, 2019. Markets had expected inflation to average 1.66 percent over the past five years, five years ago.
